# Diffusion Example

In [ ]:
# dependencies
using LFAToolkit
using LinearAlgebra
using Pkg
Pkg.activate("./")
Pkg.instantiate()
using Plots

In [ ]:
# setup
p = 3
dimension = 1

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

# operator
diffusion = GalleryOperator("diffusion", p, p, mesh)

# Jacobi smoother
jacobi = Jacobi(diffusion)

In [ ]:
# compute full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -(p - 1)π
θ_max = (p - 1)π

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # compute
    maxeigenvalues = zeros(numberruns)

    for i in 1:numberruns
        θ = [θ_min + (θ_max - θ_min)*i/numberruns]
        θ_plotting[i] = θ[1]/π
        A = computesymbols(diffusion, θ)
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i] = max(eigenvalues...)
        maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i])
    end

    # plot
    println("max eigenvalue: ", maxeigenvalue)
    xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
    plot(xrange, maxeigenvalues, title="Diffusion Symbols")
    ylims!(0.0, max(maxeigenvalues...))
# -- 2D --
elseif dimension == 2
    # compute
    θ_plotting_x = zeros(numberruns)
    θ_plotting_y = zeros(numberruns)
    maxeigenvalues = zeros(numberruns, numberruns)

    for i in 1:numberruns, j in 1:numberruns
        θ = [
            θ_min + (θ_max - θ_min)*i/numberruns,
            θ_min + (θ_max - θ_min)*j/numberruns
        ]
        A = computesymbols(diffusion, θ)
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i, j] = max(eigenvalues...)
        maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
    end

    # plot
    println("max eigenvalue: ", maxeigenvalue)
    heatmap(maxeigenvalues, title="Diffusion Symbols", transpose=true, aspect_ratio=:equal)
    xlims!(1, numberruns)
    ylims!(1, numberruns)
end